In [19]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
import torch
import torch.nn.functional as F

def compute_affinity_matrix(X, Ks, a):
    ns = X.shape[0]
    W = np.zeros((ns, ns))
     # σ^2の計算
    sigma_squared = (a / (ns * Ks)) * np.sum(
        [np.sum(np.linalg.norm(X[i] - X[j])**2) for i in range(ns) for j in np.argsort(np.linalg.norm(X[i] - X, axis=1))[1:Ks+1]]
    )
    
    for i in range(ns):
        distances = np.linalg.norm(X[i] - X, axis=1)
        nearest_indices = np.argsort(distances)[1:Ks+1]
        #print(nearest_indices)
        for j in nearest_indices:
            W[i, j] = np.exp(-distances[j]**2 / (sigma_squared))
    
    return W

def agglomerative_clustering(X, n_clusters,a):
    # アフィニティ行列を計算する
    Ks = 10  # 近傍数
    W = compute_affinity_matrix(X, Ks,a)
    
    # 凝集型クラスタリングを実行する
    Z = linkage(W, method='average')
    cluster_labels = fcluster(Z, n_clusters, criterion='maxclust')
    
    return cluster_labels

def compute_loss(X, clusters, W, Kc, λ):
    loss = 0
    for Ci in clusters:
        for x_i in Ci:
            N_Kc_Ci = torch.argsort(W[x_i])[-Kc:]  # Top Kc nearest clusters
            nearest_cluster = N_Kc_Ci[0]
            
            # Compute loss (7a)
            loss_7a = -W[x_i, nearest_cluster]
            
            # Compute loss (7b)
            loss_7b = 0
            for k in range(1, Kc):
                loss_7b += (W[x_i, nearest_cluster] - W[x_i, N_Kc_Ci[k]])
            loss_7b = -λ / (Kc - 1) * loss_7b
            
            loss += loss_7a + loss_7b
    
    return loss

# 使用例
X = np.random.rand(100, 50)  # 100サンプル、各50特徴量
print(X)
n_clusters = 5
a = 1.0  # パラメータa
labels = agglomerative_clustering(X, n_clusters,a)
print(labels)


[[0.31865705 0.4338777  0.21977522 ... 0.89287638 0.03629183 0.7796883 ]
 [0.47630695 0.09403516 0.53630307 ... 0.7681944  0.59515131 0.77691244]
 [0.15424405 0.9565707  0.46636573 ... 0.91019906 0.45007233 0.53799707]
 ...
 [0.63843677 0.60129168 0.25473161 ... 0.53179527 0.33739622 0.10772266]
 [0.74774086 0.30308011 0.69932211 ... 0.90903129 0.17290222 0.0379777 ]
 [0.23531209 0.67237018 0.63000951 ... 0.1058342  0.4084622  0.7980099 ]]
[4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 5 4 4 3 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 3]


In [22]:
# 使用例
input_size = 320  # 入力ベクトルの次元
hidden_size = 128  # 隠れ層のサイズ
output_size = 10   # クラスタ数（出力サイズ）

# 例の入力データ
X = torch.randn(50, input_size)  # 50個の320次元ベクトル
# 使用例
#X = np.random.rand(100, 50)  # 100サンプル、各50特徴量
Ks = 5  # 近傍数
a = 1.0  # パラメータa

# クラスタ (ここでは仮に設定)
clusters = [list(range(10)), list(range(10, 20)), list(range(20, 30)), list(range(30, 40)), list(range(40, 50))]

# アフィニティ行列を計算
W = compute_affinity_matrix(X, Ks, a)

# 損失を計算 (式 7a, 7b)
Kc = 5
λ = 1.0
loss = compute_loss(X, clusters, W, Kc, λ)

print(f"損失: {loss.item()}")

TypeError: argsort(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [23]:
import torch
import torch.nn.functional as F

def compute_affinity_matrix(X, Ks, a):
    ns = X.shape[0]
    W = torch.zeros((ns, ns))
    
    for i in range(ns):
        distances = torch.norm(X[i] - X, dim=1)
        nearest_indices = torch.argsort(distances)[1:Ks+1]
        for j in nearest_indices:
            W[i, j] = torch.exp(-distances[j]**2 / (a * ns * Ks))
    
    return W

def compute_loss(X, clusters, W, Kc, λ):
    loss = 0
    for Ci in clusters:
        for x_i in Ci:
            N_Kc_Ci = torch.argsort(W[x_i])[-Kc:]  # Top Kc nearest clusters
            nearest_cluster = N_Kc_Ci[0]
            
            # Compute loss (7a)
            loss_7a = -W[x_i, nearest_cluster]
            
            # Compute loss (7b)
            loss_7b = 0
            for k in range(1, Kc):
                loss_7b += (W[x_i, nearest_cluster] - W[x_i, N_Kc_Ci[k]])
            loss_7b = -λ / (Kc - 1) * loss_7b
            
            loss += loss_7a + loss_7b
    
    return loss

# 使用例
input_size = 320  # 入力ベクトルの次元
hidden_size = 128  # 隠れ層のサイズ
output_size = 10   # クラスタ数（出力サイズ）

# 例の入力データ
X = torch.randn(50, input_size)  # 50個の320次元ベクトル
Ks = 5  # 近傍数
a = 1.0  # パラメータa

# クラスタ (ここでは仮に設定)
clusters = [list(range(10)), list(range(10, 20)), list(range(20, 30)), list(range(30, 40)), list(range(40, 50))]

# アフィニティ行列を計算
W = compute_affinity_matrix(X, Ks, a)

# 損失を計算 (式 7a, 7b)
Kc = 5
λ = 1.0
loss = compute_loss(X, clusters, W, Kc, λ)

print(f"損失: {loss.item()}")


損失: -4.44038724899292


In [24]:
import torch
import torch.nn.functional as F

# 式(3)と式(4)に基づくアフィニティ行列の計算
def compute_affinity_matrix(X, Ks, a):
    ns = X.shape[0]
    W = torch.zeros((ns, ns))
    for i in range(ns):
        distances = torch.norm(X[i] - X, dim=1)
        nearest_indices = torch.argsort(distances)[1:Ks+1]
        for j in nearest_indices:
            W[i, j] = torch.exp(-distances[j]**2 / (a * ns * Ks))
    return W

# 式(5)に基づくCNNとRNNの定義
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.relu(self.i2h(combined))
        output = self.softmax(self.i2o(hidden))
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

# 累積損失の計算 (式6)
def compute_loss(outputs, targets, criterion):
    loss = 0
    for output, target in zip(outputs, targets):
        loss += criterion(output, target.unsqueeze(0))
    return loss

# クラスタ間の損失の計算 (式7a, 7b)
def compute_cluster_loss(X, clusters, W, Kc, λ):
    loss = 0
    for Ci in clusters:
        for x_i in Ci:
            N_Kc_Ci = torch.argsort(W[x_i])[-Kc:]  # Top Kc nearest clusters
            nearest_cluster = N_Kc_Ci[0]
            
            # Compute loss (7a)
            loss_7a = -W[x_i, nearest_cluster]
            
            # Compute loss (7b)
            loss_7b = 0
            for k in range(1, Kc):
                loss_7b += (W[x_i, nearest_cluster] - W[x_i, N_Kc_Ci[k]])
            loss_7b = -λ / (Kc - 1) * loss_7b
            
            loss += loss_7a + loss_7b
    
    return loss


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster

# 式(3)と式(4)に基づくアフィニティ行列の計算
def compute_affinity_matrix(X, Ks, a):
    ns = X.shape[0]
    W = torch.zeros((ns, ns))
    for i in range(ns):
        distances = torch.norm(X[i] - X, dim=1)
        nearest_indices = torch.argsort(distances)[1:Ks+1]
        for j in nearest_indices:
            W[i, j] = torch.exp(-distances[j]**2 / (a * ns * Ks))
    return W

# 式(5)に基づくCNNとRNNの定義
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.relu(self.i2h(combined))
        output = self.softmax(self.i2o(hidden))
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

# 累積損失の計算 (式6)
def compute_loss(outputs, targets, criterion):
    loss = 0
    for output, target in zip(outputs, targets):
        loss += criterion(output, target.unsqueeze(0))
    return loss

# クラスタ間の損失の計算 (式7a, 7b)
def compute_cluster_loss(X, clusters, W, Kc, λ):
    loss = 0
    for Ci in clusters:
        for x_i in Ci:
            N_Kc_Ci = torch.argsort(W[x_i])[-Kc:]  # Top Kc nearest clusters
            nearest_cluster = N_Kc_Ci[0]
            
            # Compute loss (7a)
            loss_7a = -W[x_i, nearest_cluster]
            
            # Compute loss (7b)
            loss_7b = 0
            for k in range(1, Kc):
                loss_7b += (W[x_i, nearest_cluster] - W[x_i, N_Kc_Ci[k]])
            loss_7b = -λ / (Kc - 1) * loss_7b
            
            loss += loss_7a + loss_7b
    
    return loss

# 凝集型クラスタリングに基づくクラスタ割り当ての更新 (式11)
def update_clusters(X, n_clusters, a):
    Ks = 10  # 近傍数
    W = compute_affinity_matrix(X, Ks, a)
    Z = linkage(W.numpy(), method='average')
    cluster_labels = fcluster(Z, n_clusters, criterion='maxclust')
    clusters = [[] for _ in range(n_clusters)]
    for idx, label in enumerate(cluster_labels):
        clusters[label - 1].append(idx)
    return clusters

# アルゴリズム1に基づく最適化
def joint_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_iterations, num_clusters):
    optimizer = torch.optim.Adam(list(fcnn.parameters()) + list(rnn.parameters()))

    for iteration in range(num_iterations):
        # ステップ1: 深層表現を取得 (式5a)
        X_t = fcnn(X)

        # ステップ2: 初期隠れ状態を生成
        batch_size = X.size(0)
        hidden = rnn.init_hidden(1)  # 修正：初期隠れ状態はバッチサイズ1で初期化

        # ステップ3: 時系列ごとの出力を収集
        outputs = []
        for t in range(batch_size):
            output, hidden = rnn(X_t[t].unsqueeze(0), hidden)
            outputs.append(output)

        # ステップ4: アフィニティ行列を計算 (式3, 4)
        W = compute_affinity_matrix(X, Ks, a)

        # ステップ5: クラスタ割り当てを更新 (式11)
        clusters = update_clusters(X.detach(), num_clusters, a)
        
        # ターゲットラベルを取得 (クラスタリングのラベルを仮定)
        targets = torch.tensor([label for cluster in clusters for label in cluster])

        # ステップ6: 累積損失を計算 (式6)
        time_step_loss = compute_loss(outputs, targets, criterion)

        # ステップ7: クラスタ損失を計算 (式7)
        cluster_loss = compute_cluster_loss(X, clusters, W, Kc, λ)

        # ステップ8: 総合損失を計算 (式8)
        total_loss = time_step_loss + cluster_loss

        # ステップ9: パラメータの更新
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        print(f"Iteration {iteration + 1}/{num_iterations}, Total Loss: {total_loss.item()}")

    return fcnn, rnn

# 使用例
input_size = 320  # 入力ベクトルの次元
hidden_size = 128  # 隠れ層のサイズ
output_size = 5    # クラスタ数（出力サイズ）
num_iterations = 100
num_clusters = 5

fcnn = FCNN(input_size, hidden_size)
rnn = RNN(input_size=hidden_size, hidden_size=hidden_size, output_size=output_size)
criterion = nn.CrossEntropyLoss()

# 例の入力データ
X = torch.randn(50, input_size)  # 50個の320次元ベクトル
Ks = 5  # 近傍数
a = 1.0  # パラメータa
Kc = 5
λ = 1.0

# アルゴリズム1の実行
fcnn, rnn = joint_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_iterations, num_clusters)


IndexError: Target 37 is out of bounds.

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster

# 式(3)と式(4)に基づくアフィニティ行列の計算
def compute_affinity_matrix(X, Ks, a):
    ns = X.shape[0]
    W = torch.zeros((ns, ns))
    for i in range(ns):
        distances = torch.norm(X[i] - X, dim=1)
        nearest_indices = torch.argsort(distances)[1:Ks+1]
        for j in nearest_indices:
            W[i, j] = torch.exp(-distances[j]**2 / (a * ns * Ks))
    return W

# 式(5)に基づくCNNとRNNの定義
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.relu(self.i2h(combined))
        output = self.softmax(self.i2o(hidden))
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

# 累積損失の計算 (式6)
def compute_loss(outputs, targets, criterion):
    loss = 0
    for output, target in zip(outputs, targets):
        loss += criterion(output, target.unsqueeze(0))
    return loss

# クラスタ間の損失の計算 (式7a, 7b)
def compute_cluster_loss(X, clusters, W, Kc, λ):
    loss = 0
    for Ci in clusters:
        for x_i in Ci:
            N_Kc_Ci = torch.argsort(W[x_i])[-Kc:]  # Top Kc nearest clusters
            nearest_cluster = N_Kc_Ci[0]
            
            # Compute loss (7a)
            loss_7a = -W[x_i, nearest_cluster]
            
            # Compute loss (7b)
            loss_7b = 0
            for k in range(1, Kc):
                loss_7b += (W[x_i, nearest_cluster] - W[x_i, N_Kc_Ci[k]])
            loss_7b = -λ / (Kc - 1) * loss_7b
            
            loss += loss_7a + loss_7b
    
    return loss

# 凝集型クラスタリングに基づくクラスタ割り当ての更新 (式11)
def update_clusters(X, n_clusters, a):
    Ks = 10  # 近傍数
    W = compute_affinity_matrix(X, Ks, a)
    Z = linkage(W.numpy(), method='average')
    cluster_labels = fcluster(Z, n_clusters, criterion='maxclust')
    clusters = [[] for _ in range(n_clusters)]
    for idx, label in enumerate(cluster_labels):
        clusters[label - 1].append(idx)
    return clusters, cluster_labels - 1  # クラスタラベルは0始まりにする

# アルゴリズム1に基づく最適化
def joint_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_iterations, num_clusters):
    optimizer = torch.optim.Adam(list(fcnn.parameters()) + list(rnn.parameters()))

    for iteration in range(num_iterations):
        # ステップ1: 深層表現を取得 (式5a)
        X_t = fcnn(X)

        # ステップ2: 初期隠れ状態を生成
        batch_size = X.size(0)
        hidden = rnn.init_hidden(1)  # 修正：初期隠れ状態はバッチサイズ1で初期化

        # ステップ3: 時系列ごとの出力を収集
        outputs = []
        for t in range(batch_size):
            output, hidden = rnn(X_t[t].unsqueeze(0), hidden)
            outputs.append(output)

        # ステップ4: アフィニティ行列を計算 (式3, 4)
        W = compute_affinity_matrix(X, Ks, a)

        # ステップ5: クラスタ割り当てを更新 (式11)
        clusters, cluster_labels = update_clusters(X.detach(), num_clusters, a)
        
        # ターゲットラベルを取得 (クラスタリングのラベルを仮定)
        targets = torch.tensor(cluster_labels, dtype=torch.long)

        # ステップ6: 累積損失を計算 (式6)
        time_step_loss = compute_loss(outputs, targets, criterion)

        # ステップ7: クラスタ損失を計算 (式7)
        cluster_loss = compute_cluster_loss(X, clusters, W, Kc, λ)

        # ステップ8: 総合損失を計算 (式8)
        total_loss = time_step_loss + cluster_loss

        # ステップ9: パラメータの更新
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        print(f"Iteration {iteration + 1}/{num_iterations}, Total Loss: {total_loss.item()}")

    return fcnn, rnn

# 使用例
input_size = 320  # 入力ベクトルの次元
hidden_size = 128  # 隠れ層のサイズ
output_size = 5    # クラスタ数（出力サイズ）
num_iterations = 100
num_clusters = 5

fcnn = FCNN(input_size, hidden_size)
rnn = RNN(input_size=hidden_size, hidden_size=hidden_size, output_size=output_size)
criterion = nn.CrossEntropyLoss()

# 例の入力データ
X = torch.randn(50, input_size)  # 50個の320次元ベクトル
Ks = 5  # 近傍数
a = 1.0  # パラメータa
Kc = 5
λ = 1.0

# アルゴリズム1の実行
fcnn, rnn = joint_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_iterations, num_clusters)


Iteration 1/100, Total Loss: 76.28885650634766
Iteration 2/100, Total Loss: 72.27485656738281
Iteration 3/100, Total Loss: 68.57183837890625
Iteration 4/100, Total Loss: 64.54376983642578
Iteration 5/100, Total Loss: 59.83450698852539
Iteration 6/100, Total Loss: 54.423988342285156
Iteration 7/100, Total Loss: 48.74696350097656
Iteration 8/100, Total Loss: 44.14325714111328
Iteration 9/100, Total Loss: 42.14908218383789
Iteration 10/100, Total Loss: 40.39509582519531
Iteration 11/100, Total Loss: 36.87460708618164
Iteration 12/100, Total Loss: 32.56963348388672
Iteration 13/100, Total Loss: 28.73149871826172
Iteration 14/100, Total Loss: 25.6488037109375
Iteration 15/100, Total Loss: 22.779682159423828
Iteration 16/100, Total Loss: 19.77930450439453
Iteration 17/100, Total Loss: 16.701156616210938
Iteration 18/100, Total Loss: 13.72177791595459
Iteration 19/100, Total Loss: 10.99393081665039
Iteration 20/100, Total Loss: 8.526632308959961
Iteration 21/100, Total Loss: 6.307470321655273

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster

# 式(3)と式(4)に基づくアフィニティ行列の計算
def compute_affinity_matrix(X, Ks, a):
    ns = X.shape[0]
    W = torch.zeros((ns, ns))
    for i in range(ns):
        distances = torch.norm(X[i] - X, dim=1)
        nearest_indices = torch.argsort(distances)[1:Ks+1]
        for j in nearest_indices:
            W[i, j] = torch.exp(-distances[j]**2 / (a * ns * Ks))
    return W

# 式(5)に基づくCNNとRNNの定義
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.relu(self.i2h(combined))
        output = self.softmax(self.i2o(hidden))
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

# 累積損失の計算 (式6)
def compute_loss(outputs, targets, criterion):
    loss = 0
    for output, target in zip(outputs, targets):
        loss += criterion(output, target.unsqueeze(0))
    return loss

# クラスタ間の損失の計算 (式7a, 7b)
def compute_cluster_loss(X, clusters, W, Kc, λ):
    loss = 0
    for Ci in clusters:
        for x_i in Ci:
            N_Kc_Ci = torch.argsort(W[x_i])[-Kc:]  # Top Kc nearest clusters
            nearest_cluster = N_Kc_Ci[0]
            
            # Compute loss (7a)
            loss_7a = -W[x_i, nearest_cluster]
            
            # Compute loss (7b)
            loss_7b = 0
            for k in range(1, Kc):
                loss_7b += (W[x_i, nearest_cluster] - W[x_i, N_Kc_Ci[k]])
            loss_7b = -λ / (Kc - 1) * loss_7b
            
            loss += loss_7a + loss_7b
    
    return loss

# 凝集型クラスタリングに基づくクラスタ割り当ての更新 (式11)
def update_clusters(X, n_clusters, a):
    Ks = 10  # 近傍数
    W = compute_affinity_matrix(X, Ks, a)
    Z = linkage(W.numpy(), method='average')
    cluster_labels = fcluster(Z, n_clusters, criterion='maxclust')
    clusters = [[] for _ in range(n_clusters)]
    for idx, label in enumerate(cluster_labels):
        clusters[label - 1].append(idx)
    return clusters, cluster_labels - 1  # クラスタラベルは0始まりにする

# アルゴリズム1に基づく最適化
def joint_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_iterations, num_clusters):
    optimizer = torch.optim.Adam(list(fcnn.parameters()) + list(rnn.parameters()))
    final_clusters = None  # 最終クラスタリング結果を保存

    for iteration in range(num_iterations):
        # ステップ1: 深層表現を取得 (式5a)
        X_t = fcnn(X)

        # ステップ2: 初期隠れ状態を生成
        batch_size = X.size(0)
        hidden = rnn.init_hidden(1)  # 修正：初期隠れ状態はバッチサイズ1で初期化

        # ステップ3: 時系列ごとの出力を収集
        outputs = []
        for t in range(batch_size):
            output, hidden = rnn(X_t[t].unsqueeze(0), hidden)
            outputs.append(output)

        # ステップ4: アフィニティ行列を計算 (式3, 4)
        W = compute_affinity_matrix(X, Ks, a)

        # ステップ5: クラスタ割り当てを更新 (式11)
        clusters, cluster_labels = update_clusters(X.detach(), num_clusters, a)
        final_clusters = cluster_labels  # 最終クラスタリング結果を保存
        
        # ターゲットラベルを取得 (クラスタリングのラベルを仮定)
        targets = torch.tensor(cluster_labels, dtype=torch.long)

        # ステップ6: 累積損失を計算 (式6)
        time_step_loss = compute_loss(outputs, targets, criterion)

        # ステップ7: クラスタ損失を計算 (式7)
        cluster_loss = compute_cluster_loss(X, clusters, W, Kc, λ)

        # ステップ8: 総合損失を計算 (式8)
        total_loss = time_step_loss + cluster_loss

        # ステップ9: パラメータの更新
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        print(f"Iteration {iteration + 1}/{num_iterations}, Total Loss: {total_loss.item()}")

    return fcnn, rnn, final_clusters

# 使用例
input_size = 320  # 入力ベクトルの次元
hidden_size = 128  # 隠れ層のサイズ
output_size = 5    # クラスタ数（出力サイズ）
num_iterations = 100
num_clusters = 5

fcnn = FCNN(input_size, hidden_size)
rnn = RNN(input_size=hidden_size, hidden_size=hidden_size, output_size=output_size)
criterion = nn.CrossEntropyLoss()

# 例の入力データ
X = torch.randn(50, input_size)  # 50個の320次元ベクトル
Ks = 5  # 近傍数
a = 1.0  # パラメータa
Kc = 5
λ = 1.0

# アルゴリズム1の実行
fcnn, rnn, final_clusters = joint_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_iterations, num_clusters)

# クラスタリング結果を表示
print("最終クラスタリング結果:", final_clusters)


Iteration 1/100, Total Loss: 77.10551452636719
Iteration 2/100, Total Loss: 73.82721710205078
Iteration 3/100, Total Loss: 71.30651092529297
Iteration 4/100, Total Loss: 68.907470703125
Iteration 5/100, Total Loss: 66.2588882446289
Iteration 6/100, Total Loss: 63.071083068847656
Iteration 7/100, Total Loss: 59.18975830078125
Iteration 8/100, Total Loss: 54.600196838378906
Iteration 9/100, Total Loss: 49.40264129638672
Iteration 10/100, Total Loss: 44.141056060791016
Iteration 11/100, Total Loss: 39.61823654174805
Iteration 12/100, Total Loss: 35.71752166748047
Iteration 13/100, Total Loss: 31.749624252319336
Iteration 14/100, Total Loss: 27.48515510559082
Iteration 15/100, Total Loss: 22.59560203552246
Iteration 16/100, Total Loss: 17.604249954223633
Iteration 17/100, Total Loss: 13.344947814941406
Iteration 18/100, Total Loss: 9.96843147277832
Iteration 19/100, Total Loss: 7.280675411224365
Iteration 20/100, Total Loss: 5.061440944671631
Iteration 21/100, Total Loss: 3.12581729888916


In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import silhouette_score

# 式(3)と式(4)に基づくアフィニティ行列の計算
def compute_affinity_matrix(X, Ks, a):
    ns = X.shape[0]
    W = torch.zeros((ns, ns))
    for i in range(ns):
        distances = torch.norm(X[i] - X, dim=1)
        nearest_indices = torch.argsort(distances)[1:Ks+1]
        for j in nearest_indices:
            W[i, j] = torch.exp(-distances[j]**2 / (a * ns * Ks))
    return W

# 式(5)に基づくCNNとRNNの定義
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.relu(self.i2h(combined))
        output = self.softmax(self.i2o(hidden))
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

# 累積損失の計算 (式6)
def compute_loss(outputs, targets, criterion):
    loss = 0
    for output, target in zip(outputs, targets):
        loss += criterion(output, target.unsqueeze(0))
    return loss

# クラスタ間の損失の計算 (式7a, 7b)
def compute_cluster_loss(X, clusters, W, Kc, λ):
    loss = 0
    for Ci in clusters:
        for x_i in Ci:
            N_Kc_Ci = torch.argsort(W[x_i])[-Kc:]  # Top Kc nearest clusters
            nearest_cluster = N_Kc_Ci[0]
            
            # Compute loss (7a)
            loss_7a = -W[x_i, nearest_cluster]
            
            # Compute loss (7b)
            loss_7b = 0
            for k in range(1, Kc):
                loss_7b += (W[x_i, nearest_cluster] - W[x_i, N_Kc_Ci[k]])
            loss_7b = -λ / (Kc - 1) * loss_7b
            
            loss += loss_7a + loss_7b
    
    return loss

# 凝集型クラスタリングに基づくクラスタ割り当ての更新 (式11)
def update_clusters(X, n_clusters, a):
    Ks = 10  # 近傍数
    W = compute_affinity_matrix(X, Ks, a)
    Z = linkage(W.cpu().numpy(), method='average')  # .cpu()を追加してGPUからCPUに移動
    cluster_labels = fcluster(Z, n_clusters, criterion='maxclust')
    clusters = [[] for _ in range(n_clusters)]
    for idx, label in enumerate(cluster_labels):
        clusters[label - 1].append(idx)
    return clusters, cluster_labels - 1  # クラスタラベルは0始まりにする

# 最適なクラスタ数を決定
def determine_optimal_clusters(X, max_clusters, a):
    best_num_clusters = 2
    best_silhouette_score = -1
    for n_clusters in range(2, max_clusters + 1):
        clusters, cluster_labels = update_clusters(X, n_clusters, a)
        if len(set(cluster_labels)) == 1:  # クラスタが一つに収束した場合はスキップ
            continue
        score = silhouette_score(X.cpu().numpy(), cluster_labels)  # .cpu()を追加してGPUからCPUに移動
        if score > best_silhouette_score:
            best_silhouette_score = score
            best_num_clusters = n_clusters
    return best_num_clusters

# アルゴリズム1に基づく最適化
def joint_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_iterations, max_clusters):
    final_clusters = None  # 最終クラスタリング結果を保存

    # 最適なクラスタ数を決定
    #num_clusters = determine_optimal_clusters(X, max_clusters, a)
    num_clusters =5
    print(f"Optimal number of clusters determined: {num_clusters}")

    # RNNの出力サイズを更新
    rnn.i2o = nn.Linear(rnn.hidden_size, num_clusters)
    rnn.softmax = nn.LogSoftmax(dim=1)

    optimizer = torch.optim.Adam(list(fcnn.parameters()) + list(rnn.parameters()))

    for iteration in range(num_iterations):
        # ステップ1: 深層表現を取得 (式5a)
        X_t = fcnn(X)

        # ステップ2: 初期隠れ状態を生成
        batch_size = X.size(0)
        hidden = rnn.init_hidden(1)  # 修正：初期隠れ状態はバッチサイズ1で初期化

        # ステップ3: 時系列ごとの出力を収集
        outputs = []
        for t in range(batch_size):
            output, hidden = rnn(X_t[t].unsqueeze(0), hidden)
            outputs.append(output)

        # ステップ4: アフィニティ行列を計算 (式3, 4)
        W = compute_affinity_matrix(X, Ks, a)

        # ステップ5: クラスタ割り当てを更新 (式11)
        clusters, cluster_labels = update_clusters(X.detach(), num_clusters, a)
        final_clusters = cluster_labels  # 最終クラスタリング結果を保存
        
        # ターゲットラベルを取得 (クラスタリングのラベルを仮定)
        targets = torch.tensor(cluster_labels, dtype=torch.long)

        # ステップ6: 累積損失を計算 (式6)
        time_step_loss = compute_loss(outputs, targets, criterion)

        # ステップ7: クラスタ損失を計算 (式7)
        cluster_loss = compute_cluster_loss(X, clusters, W, Kc, λ)

        # ステップ8: 総合損失を計算 (式8)
        total_loss = time_step_loss + cluster_loss

        # ステップ9: パラメータの更新
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        print(f"Iteration {iteration + 1}/{num_iterations}, Total Loss: {total_loss.item()}")

    return fcnn, rnn, final_clusters

# 使用例
input_size = 320  # 入力ベクトルの次元
hidden_size = 128  # 隠れ層のサイズ
max_clusters = 10  # 最大クラスタ数
num_iterations = 100

fcnn = FCNN(input_size, hidden_size)
# 出力サイズは動的に設定されるため、初期値は仮設定
rnn = RNN(input_size=hidden_size, hidden_size=hidden_size, output_size=max_clusters)
criterion = nn.CrossEntropyLoss()

# 例の入力データ
X = torch.randn(50, input_size)  # 50個の320次元ベクトル
Ks = 5  # 近傍数
a = 1.0  # パラメータa
Kc = 5
λ = 1.0

# アルゴリズム1の実行
fcnn, rnn, final_clusters = joint_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_iterations, max_clusters)

# クラスタリング結果を表示
print("最終クラスタリング結果:", final_clusters)


Optimal number of clusters determined: 5
Iteration 1/100, Total Loss: 74.16905975341797
Iteration 2/100, Total Loss: 69.88932037353516
Iteration 3/100, Total Loss: 65.56550598144531
Iteration 4/100, Total Loss: 60.562686920166016
Iteration 5/100, Total Loss: 54.550880432128906
Iteration 6/100, Total Loss: 47.29065704345703
Iteration 7/100, Total Loss: 38.99746322631836
Iteration 8/100, Total Loss: 31.132801055908203
Iteration 9/100, Total Loss: 27.076061248779297
Iteration 10/100, Total Loss: 28.47885513305664
Iteration 11/100, Total Loss: 28.875816345214844
Iteration 12/100, Total Loss: 26.326587677001953
Iteration 13/100, Total Loss: 22.42275619506836
Iteration 14/100, Total Loss: 18.799991607666016
Iteration 15/100, Total Loss: 16.314483642578125
Iteration 16/100, Total Loss: 14.936219215393066
Iteration 17/100, Total Loss: 14.192179679870605
Iteration 18/100, Total Loss: 13.52741527557373
Iteration 19/100, Total Loss: 12.675494194030762
Iteration 20/100, Total Loss: 11.589444160461

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import silhouette_score

# 式(3)と式(4)に基づくアフィニティ行列の計算
def compute_affinity_matrix(X, Ks, a):
    ns = X.shape[0]
    W = torch.zeros((ns, ns))
    for i in range(ns):
        distances = torch.norm(X[i] - X, dim=1)
        nearest_indices = torch.argsort(distances)[1:Ks+1]
        for j in nearest_indices:
            W[i, j] = torch.exp(-distances[j]**2 / (a * ns * Ks))
    return W

# 式(5)に基づくCNNとRNNの定義
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.relu(self.i2h(combined))
        output = self.softmax(self.i2o(hidden))
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

# 累積損失の計算 (式6)
def compute_loss(outputs, targets, criterion):
    loss = 0
    for output, target in zip(outputs, targets):
        loss += criterion(output, target.unsqueeze(0))
    return loss

# クラスタ間の損失の計算 (式7a, 7b)
def compute_cluster_loss(X, clusters, W, Kc, λ):
    loss = 0
    for Ci in clusters:
        for x_i in Ci:
            N_Kc_Ci = torch.argsort(W[x_i])[-Kc:]  # Top Kc nearest clusters
            nearest_cluster = N_Kc_Ci[0]
            
            # Compute loss (7a)
            loss_7a = -W[x_i, nearest_cluster]
            
            # Compute loss (7b)
            loss_7b = 0
            for k in range(1, Kc):
                loss_7b += (W[x_i, nearest_cluster] - W[x_i, N_Kc_Ci[k]])
            loss_7b = -λ / (Kc - 1) * loss_7b
            
            loss += loss_7a + loss_7b
    
    return loss

# 凝集型クラスタリングに基づくクラスタ割り当ての更新 (式11)
def update_clusters(X, n_clusters, a):
    Ks = 10  # 近傍数
    W = compute_affinity_matrix(X, Ks, a)
    Z = linkage(W.cpu().numpy(), method='average')  # .cpu()を追加してGPUからCPUに移動
    cluster_labels = fcluster(Z, n_clusters, criterion='maxclust')
    clusters = [[] for _ in range(n_clusters)]
    for idx, label in enumerate(cluster_labels):
        clusters[label - 1].append(idx)
    return clusters, cluster_labels - 1  # クラスタラベルは0始まりにする

# # 最適なクラスタ数を決定
# def determine_optimal_clusters(X, max_clusters, a):
#     best_num_clusters = 2
#     best_silhouette_score = -1
#     for n_clusters in range(2, max_clusters + 1):
#         clusters, cluster_labels = update_clusters(X, n_clusters, a)
#         if len(set(cluster_labels)) == 1:  # クラスタが一つに収束した場合はスキップ
#             continue
#         score = silhouette_score(X.cpu().numpy(), cluster_labels)  # .cpu()を追加してGPUからCPUに移動
#         if score > best_silhouette_score:
#             best_silhouette_score = score
#             best_num_clusters = n_clusters
#     return best_num_clusters

# 部分展開の最適化
def partial_unrolling_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_periods, unrolling_rate, max_clusters):
    final_clusters = None  # 最終クラスタリング結果を保存

    # 初期のクラスタ数
    num_clusters = 5
    print(f"Initial number of clusters: {num_clusters}")

    optimizer = torch.optim.Adam(list(fcnn.parameters()) + list(rnn.parameters()))

    period_length = lambda ns_c: int(np.ceil(unrolling_rate * ns_c))

    for period in range(num_periods):
        current_clusters, cluster_labels = update_clusters(X, num_clusters, a)
        num_clusters = len(current_clusters)  # 現在のクラスタ数を更新

        # RNNの出力サイズを更新
        rnn.i2o = nn.Linear(rnn.hidden_size, num_clusters)
        rnn.softmax = nn.LogSoftmax(dim=1)

        # 順伝播：クラスタをマージ
        total_loss = 0
        for step in range(period_length(num_clusters)):
            # ステップ1: 深層表現を取得 (式5a)
            X_t = fcnn(X)

            # ステップ2: 初期隠れ状態を生成
            batch_size = X.size(0)
            hidden = rnn.init_hidden(1)  # 修正：初期隠れ状態はバッチサイズ1で初期化

            # ステップ3: 時系列ごとの出力を収集
            outputs = []
            for t in range(batch_size):
                output, hidden = rnn(X_t[t].unsqueeze(0), hidden)
                outputs.append(output)

            # ステップ4: アフィニティ行列を計算 (式3, 4)
            W = compute_affinity_matrix(X, Ks, a)

            # ステップ5: クラスタ割り当てを更新 (式11)
            clusters, cluster_labels = update_clusters(X.detach(), num_clusters, a)
            final_clusters = cluster_labels  # 最終クラスタリング結果を保存
            
            # ターゲットラベルを取得 (クラスタリングのラベルを仮定)
            targets = torch.tensor(cluster_labels, dtype=torch.long)

            # ステップ6: 累積損失を計算 (式6)
            time_step_loss = compute_loss(outputs, targets, criterion)

            # ステップ7: クラスタ損失を計算 (式7)
            cluster_loss = compute_cluster_loss(X, clusters, W, Kc, λ)

            # ステップ8: 総合損失を計算 (式8)
            total_loss += time_step_loss + cluster_loss

            print(f"Period {period + 1}/{num_periods}, Step {step + 1}/{period_length(num_clusters)}, Total Loss: {total_loss.item()}")

        # 逆伝播：CNNパラメータを更新
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

    return fcnn, rnn, final_clusters

# 使用例
input_size = 320  # 入力ベクトルの次元
hidden_size = 128  # 隠れ層のサイズ
max_clusters = 10  # 最大クラスタ数
num_periods = 5
unrolling_rate = 0.9  # 部分展開の割合

fcnn = FCNN(input_size, hidden_size)
rnn = RNN(input_size=hidden_size, hidden_size=hidden_size, output_size=max_clusters)
criterion = nn.CrossEntropyLoss()

# 例の入力データ
X = torch.randn(50, input_size)  # 50個の320次元ベクトル
Ks = 5  # 近傍数
a = 1.0  # パラメータa
Kc = 5
λ = 1.0

# 部分展開最適化の実行
fcnn, rnn, final_clusters = partial_unrolling_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_periods, unrolling_rate, max_clusters)

# クラスタリング結果を表示
print("最終クラスタリング結果:", final_clusters)


Initial number of clusters: 5
Period 1/5, Step 1/5, Total Loss: 77.14836883544922
Period 1/5, Step 2/5, Total Loss: 154.29673767089844
Period 1/5, Step 3/5, Total Loss: 231.44509887695312
Period 1/5, Step 4/5, Total Loss: 308.5934753417969
Period 1/5, Step 5/5, Total Loss: 385.7418518066406
Period 2/5, Step 1/5, Total Loss: 74.82903289794922
Period 2/5, Step 2/5, Total Loss: 149.65806579589844
Period 2/5, Step 3/5, Total Loss: 224.48709106445312
Period 2/5, Step 4/5, Total Loss: 299.3161315917969
Period 2/5, Step 5/5, Total Loss: 374.1451721191406
Period 3/5, Step 1/5, Total Loss: 76.27616882324219
Period 3/5, Step 2/5, Total Loss: 152.55233764648438
Period 3/5, Step 3/5, Total Loss: 228.82850646972656
Period 3/5, Step 4/5, Total Loss: 305.10467529296875
Period 3/5, Step 5/5, Total Loss: 381.380859375
Period 4/5, Step 1/5, Total Loss: 74.69186401367188
Period 4/5, Step 2/5, Total Loss: 149.38372802734375
Period 4/5, Step 3/5, Total Loss: 224.07559204101562
Period 4/5, Step 4/5, Total L

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import silhouette_score
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

# 式(3)と式(4)に基づくアフィニティ行列の計算
def compute_affinity_matrix(X, Ks, a):
    ns = X.shape[0]
    W = torch.zeros((ns, ns))
    for i in range(ns):
        distances = torch.norm(X[i] - X, dim=1)
        nearest_indices = torch.argsort(distances)[1:Ks+1]
        for j in nearest_indices:
            W[i, j] = torch.exp(-distances[j]**2 / (a * ns * Ks))
    return W

# 式(5)に基づくCNNとRNNの定義
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.relu(self.i2h(combined))
        output = self.softmax(self.i2o(hidden))
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

# 累積損失の計算 (式6)
def compute_loss(outputs, targets, criterion):
    loss = 0
    for output, target in zip(outputs, targets):
        loss += criterion(output, target.unsqueeze(0))
    return loss

# クラスタ間の損失の計算 (式7a, 7b)
def compute_cluster_loss(X, clusters, W, Kc, λ):
    loss = 0
    for Ci in clusters:
        for x_i in Ci:
            N_Kc_Ci = torch.argsort(W[x_i])[-Kc:]  # Top Kc nearest clusters
            nearest_cluster = N_Kc_Ci[0]
            
            # Compute loss (7a)
            loss_7a = -W[x_i, nearest_cluster]
            
            # Compute loss (7b)
            loss_7b = 0
            for k in range(1, Kc):
                loss_7b += (W[x_i, nearest_cluster] - W[x_i, N_Kc_Ci[k]])
            loss_7b = -λ / (Kc - 1) * loss_7b
            
            loss += loss_7a + loss_7b
    
    return loss

# 凝集型クラスタリングに基づくクラスタ割り当ての更新 (式11)
def update_clusters(X, n_clusters, a):
    Ks = 10  # 近傍数
    W = compute_affinity_matrix(X, Ks, a)
    Z = linkage(W.cpu().numpy(), method='average')  # .cpu()を追加してGPUからCPUに移動
    cluster_labels = fcluster(Z, n_clusters, criterion='maxclust')
    clusters = [[] for _ in range(n_clusters)]
    for idx, label in enumerate(cluster_labels):
        clusters[label - 1].append(idx)
    return clusters, cluster_labels - 1  # クラスタラベルは0始まりにする

# クラスラベルの更新
def update_labels(features, label_pre, Ks, a, iter):
    if features.shape[0] <= 1:
        return label_pre
    distances, indices, W = compute_affinity(features.numpy(), Ks, a)
    W = csr_matrix(W)
    if iter == 0:
        print("initialize clusters...")
        n_components, labels = connected_components(csgraph=W, directed=False, return_labels=True)
        label_pre = [np.where(labels == i)[0].tolist() for i in range(n_components)]
        return label_pre
    return label_pre

# ラベルのマージ
def merge_labels(network_table, epoch_reset_labels, target_nclusters_table, trainData_data, args):
    label_pre_table_table = []
    for i, model in enumerate(network_table):
        if epoch_reset_labels[i] == 0 or args.updateCNN == 0:
            features = trainData_data.float()
        else:
            features = extract_features(model, trainData_data)
        
        if args.centralize_feature == 1:
            features -= features.mean(dim=0, keepdim=True)

        if args.normalize == 1:
            features = features / features.norm(p=2, dim=1, keepdim=True)
        
        label_pre_table_table.append(update_labels(features, [], target_nclusters_table[i], epoch_reset_labels[i]))
        epoch_reset_labels[i] += 1
        label_pre_tensor_table = [cvt2_tensor_labels(labels, 1, trainData_data.size(0)) for labels in label_pre_table_table]
    return label_pre_tensor_table

# テンソルラベルへの変換
def cvt2_tensor_labels(labels, ind_s, ind_e):
    label_tensor = torch.zeros(ind_e - ind_s + 1, 1)
    for i, label_group in enumerate(labels):
        for j in label_group:
            label_tensor[j, 0] = i + 1
    return label_tensor

# アルゴリズム1に基づく最適化
def joint_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_iterations, max_clusters, args):
    final_clusters = None  # 最終クラスタリング結果を保存
    epoch_reset_labels = [0]

    # 最適なクラスタ数を決定
    num_clusters = determine_optimal_clusters(X, max_clusters, a)
    print(f"Optimal number of clusters determined: {num_clusters}")

    # RNNの出力サイズを更新
    rnn.i2o = nn.Linear(rnn.hidden_size, num_clusters)
    rnn.softmax = nn.LogSoftmax(dim=1)

    optimizer = torch.optim.Adam(list(fcnn.parameters()) + list(rnn.parameters()))

    for iteration in range(num_iterations):
        # ステップ1: 深層表現を取得 (式5a)
        X_t = fcnn(X)

        # ステップ2: 初期隠れ状態を生成
        batch_size = X.size(0)
        hidden = rnn.init_hidden(1)  # 修正：初期隠れ状態はバッチサイズ1で初期化

        # ステップ3: 時系列ごとの出力を収集
        outputs = []
        for t in range(batch_size):
            output, hidden = rnn(X_t[t].unsqueeze(0), hidden)
            outputs.append(output)

        # ステップ4: アフィニティ行列を計算 (式3, 4)
        W = compute_affinity_matrix(X, Ks, a)

        # ステップ5: クラスタ割り当てを更新 (式11)
        clusters, cluster_labels = update_clusters(X.detach(), num_clusters, a)
        final_clusters = cluster_labels  # 最終クラスタリング結果を保存
        
        # ターゲットラベルを取得 (クラスタリングのラベルを仮定)
        targets = torch.tensor(cluster_labels, dtype=torch.long)

        # ステップ6: 累積損失を計算 (式6)
        time_step_loss = compute_loss(outputs, targets, criterion)

        # ステップ7: クラスタ損失を計算 (式7)
        cluster_loss = compute_cluster_loss(X, clusters, W, Kc, λ)

        # ステップ8: 総合損失を計算 (式8)
        total_loss = time_step_loss + cluster_loss

        # ステップ9: パラメータの更新
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # クラスラベルの更新とマージ
        label_pre_tensor_table = merge_labels([fcnn, rnn], epoch_reset_labels, [num_clusters], X, args)
        print(f"Iteration {iteration + 1}/{num_iterations}, Total Loss: {total_loss.item()}")

    return fcnn, rnn, final_clusters

# 使用例
input_size = 320  # 入力ベクトルの次元
hidden_size = 128  # 隠れ層のサイズ
max_clusters = 10  # 最大クラスタ数
num_iterations = 100

fcnn = FCNN(input_size, hidden_size)
# 出力サイズは動的に設定されるため、初期値は仮設定
rnn = RNN(input_size=hidden_size, hidden_size=hidden_size, output_size=max_clusters)
criterion = nn.CrossEntropyLoss()

# 例の入力データ
X = torch.randn(50, input_size)  # 50個の320次元ベクトル
Ks = 5  # 近傍数
a = 1.0  # パラメータa
Kc = 5
λ = 1.0

# アルゴリズム1の実行
args = lambda: None
args.updateCNN = 1
args.centralize_feature = 1
args.normalize = 1
fcnn, rnn, final_clusters = joint_optimization(X, fcnn, rnn, criterion, Ks, a, Kc, λ, num_iterations, max_clusters, args)

# クラスタリング結果を表示
print("最終クラスタリング結果:", final_clusters)


Optimal number of clusters determined: 2


TypeError: update_labels() missing 1 required positional argument: 'iter'

In [ ]:
def compute_affinity(features, k_s):
    n_samples = features.shape[0]
    k_s = max(1, min(k_s, n_samples - 1))
    nbrs = NearestNeighbors(n_neighbors=k_s, algorithm='auto').fit(features)
    distances, indices = nbrs.kneighbors(features)
    
    # Create a square matrix of shape (n_samples, n_samples)
    W = np.zeros((n_samples, n_samples))
    
    # Fill the affinity matrix
    for i in range(n_samples):
        for j in range(k_s):
            W[i, indices[i, j]] = np.exp(-distances[i, j] ** 2)
    
    return distances, indices, W


In [16]:
import torch
import torch.nn as nn

class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.relu(self.i2h(combined))
        output = self.softmax(self.i2o(hidden))
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)
    
def compute_loss(outputs, targets, criterion):
    loss = 0
    for t in range(len(outputs)):
        loss += criterion(outputs[t], targets[t])
    return loss

# 使用例
input_size = 320  # 入力ベクトルの次元
hidden_size = 128  # 隠れ層のサイズ
output_size = 10   # クラスタ数（出力サイズ）

fcnn = FCNN(input_size, hidden_size)
rnn = RNN(input_size=hidden_size, hidden_size=hidden_size, output_size=output_size)
criterion = nn.CrossEntropyLoss()

# 例の入力データ
inputs = torch.randn(50, input_size)  # 50個の320次元ベクトル
targets = torch.randint(0, output_size, (50,))  # 50個のターゲットラベル
batch_size = inputs.size(0)

# 時刻 t の初期隠れ状態を生成
hidden = rnn.init_hidden(batch_size)

# 時刻 t の処理 (式 5a)
X_t = fcnn(inputs)

# 時刻 t の隠れ状態を更新 (式 5b)
outputs = []
for t in range(batch_size):
    output, hidden = rnn(X_t[t].unsqueeze(0), hidden)
    outputs.append(output)

print(f"X_t: {X_t}")
print(f"h_t: {hidden}")
print(f"y_t: {outputs}")
# 損失の計算 (式 6)
loss = compute_loss(outputs, targets, criterion)

print(f"損失: {loss.item()}")


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 50 for tensor number 1 in the list.

In [18]:
import torch
import torch.nn as nn

class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.relu(self.i2h(combined))
        output = self.softmax(self.i2o(hidden))
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

def compute_loss(outputs, targets, criterion):
    loss = 0
    for output, target in zip(outputs, targets):
        loss += criterion(output, target.unsqueeze(0))
    return loss

# 使用例
input_size = 320  # 入力ベクトルの次元
hidden_size = 128  # 隠れ層のサイズ
output_size = 10   # クラスタ数（出力サイズ）

fcnn = FCNN(input_size, hidden_size)
rnn = RNN(input_size=hidden_size, hidden_size=hidden_size, output_size=output_size)
criterion = nn.CrossEntropyLoss()

# 例の入力データ
inputs = torch.randn(50, input_size)  # 50個の320次元ベクトル
targets = torch.randint(0, output_size, (50,))  # 50個のターゲットラベル
batch_size = inputs.size(0)

# 初期隠れ状態を生成
hidden = rnn.init_hidden(batch_size)

# FCNNを通じて入力データを変換 (式 5a)
X_t = fcnn(inputs)

# タイムステップごとの出力を収集するリスト
outputs = []

# 各タイムステップでRNNを適用し、出力と隠れ状態を更新 (式 5b, 5c)
for t in range(batch_size):
    output, hidden = rnn(X_t[t].unsqueeze(0), hidden)
    outputs.append(output)

# ターゲットラベルを各タイムステップに対応する形に変換
targets = targets.unsqueeze(1).expand(-1, batch_size)

# 累積損失を計算 (式 6)
loss = compute_loss(outputs, targets, criterion)

print(f"損失: {loss.item()}")


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 50 for tensor number 1 in the list.

In [42]:
import torch

class AggClustering:
    def __init__(self, K_c):
        self.K_c = K_c

    def search_clusters(self, A_s_t):
        A_sorted, idx_sort = torch.sort(A_s_t, dim=0, descending=True)
        aff = torch.zeros(1, A_sorted.size(1), dtype=torch.float32)

        for i in range(A_sorted.size(1)):
            aff[0, i] = A_sorted[0, i].item()
            if A_sorted.size(1) > 100:
                for k in range(1, self.K_c):
                    aff[0, i] += (A_sorted[0, i] - A_sorted[k, i]).item() / (self.K_c - 1)

        v_c, idx_c = torch.max(aff, dim=1)
        idx_c_b = idx_c.item()
        idx_c_a = idx_sort[0, idx_c_b].item()

        if idx_c_a == idx_c_b:
            raise ValueError("Error: idx_c_a and idx_c_b are the same")
        elif idx_c_a > idx_c_b:
            idx_c_a, idx_c_b = idx_c_b, idx_c_a

        return idx_c_a, idx_c_b

# 使用例
K_c = 10  # K_cの例
agg_clustering = AggClustering(K_c)

A_s_t = torch.rand(50, 50)  # アフィニティマトリックスの例
idx_c_a, idx_c_b = agg_clustering.search_clusters(A_s_t)
print("idx_c_a:", idx_c_a, "idx_c_b:", idx_c_b)


idx_c_a: 14 idx_c_b: 17


In [44]:
# ベクトルデータの定義
input_size = 320  # 入力ベクトルの次元
X = torch.randn(50, input_size)  # 50個の320次元ベクトル
y = torch.randint(0, 5, (50,))  # 0から4までのランダムなラベル

print(X.size())
print(y.size())

torch.Size([50, 320])
torch.Size([50])
